In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_name_3": "category", 
            "event_type_1": "category", "event_type_2": "category", "event_type_3": "category",
            "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

In [4]:
h = 28 
max_lags = 57
tr_last = 1913 # = datetime(2016, 4, 24)
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [5]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("./sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("./exdata/calendar2.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("./sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [6]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [7]:
FIRST_DAY = 1 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [8]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 16.8 s, sys: 2.59 s, total: 19.4 s
Wall time: 19.4 s


(46027957, 25)

In [9]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [10]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_name_3,event_type_3,sell_price
0,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46
1,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46
2,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46
3,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0,0,0.46
4,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0,0,0.46


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 24 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  event_name_3  int16         
 22  event_type_3  int16         
 23  sell_price    float32       
d

In [12]:
%%time

create_fea(df)
df.shape

CPU times: user 1min 34s, sys: 4.71 s, total: 1min 39s
Wall time: 1min 39s


(46027957, 33)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 33 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  event_name_3  int16         
 22  event_type_3  int16         
 23  sell_price    float32       
 

In [14]:
df.d

0              d_1
1              d_2
2              d_3
3              d_4
4              d_5
             ...  
46027952    d_1913
46027953    d_1912
46027954    d_1913
46027955    d_1912
46027956    d_1913
Name: d, Length: 46027957, dtype: object

In [33]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_name_3,event_type_3,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,29
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,30
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0,0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,31
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0,0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0,0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,2


In [34]:
df.dropna(inplace = True)
df.shape

(44351007, 33)

In [35]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_name_3", "event_type_1", "event_type_2", "event_type_3"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [ ]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [38]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 8.49 s, sys: 400 ms, total: 8.89 s
Wall time: 8.88 s


In [39]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

66

In [40]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [41]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 3.0791
[40]	valid_0's rmse: 2.60651
[60]	valid_0's rmse: 2.48055
[80]	valid_0's rmse: 2.4445
[100]	valid_0's rmse: 2.43218
[120]	valid_0's rmse: 2.42336
[140]	valid_0's rmse: 2.41699
[160]	valid_0's rmse: 2.4113
[180]	valid_0's rmse: 2.40342
[200]	valid_0's rmse: 2.39727
[220]	valid_0's rmse: 2.39117
[240]	valid_0's rmse: 2.38763
[260]	valid_0's rmse: 2.38311
[280]	valid_0's rmse: 2.37956
[300]	valid_0's rmse: 2.37557
[320]	valid_0's rmse: 2.37064
[340]	valid_0's rmse: 2.36813
[360]	valid_0's rmse: 2.36424
[380]	valid_0's rmse: 2.36106
[400]	valid_0's rmse: 2.35775
[420]	valid_0's rmse: 2.35547
[440]	valid_0's rmse: 2.35259
[460]	valid_0's rmse: 2.35021
[480]	valid_0's rmse: 2.34849
[500]	valid_0's rmse: 2.34623
[520]	valid_0's rmse: 2.34295
[540]	valid_0's rmse: 2.34065
[560]	valid_0's rmse: 2.33811
[580]	valid_0's rmse: 2.33548
[600]	valid_0's rmse: 2.33377
[620]	valid_0's rmse: 2.33165
[640]	valid_0's rmse: 2.32942
[660]	valid_0's rmse: 2.32761
[680]	valid_0's r

In [42]:
m_lgb.save_model("model.lgb")

In [43]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
    #te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [44]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.900959,0.846993,0.843530,0.822915,1.067092,1.148474,1.237257,1.016322,0.977166,0.974565,0.971254,1.097485,1.317380,1.132391,1.015995,0.949060,0.952826,0.963400,1.085783,1.349904,1.287746,0.984722,0.878017,0.850960,0.868184,1.046077,1.299787,1.241761
1,FOODS_1_001_CA_2_validation,0.947681,1.037914,0.927550,1.323528,1.388636,1.598580,1.393178,0.973287,0.983578,0.956035,0.951097,1.189399,1.597680,1.241556,1.126212,1.068165,1.086547,1.089336,1.300449,1.757588,1.386870,1.082097,1.036707,1.015686,1.046437,1.247382,1.712865,1.416254
2,FOODS_1_001_CA_3_validation,1.098065,1.052737,0.928638,0.905167,0.959807,1.120069,1.296803,1.131207,1.130072,0.983256,0.975465,1.102550,1.383946,1.314459,1.161765,1.143278,1.057774,1.066230,1.126196,1.590009,1.698748,1.159807,1.092208,0.969167,0.958199,1.046574,1.299163,1.307476
3,FOODS_1_001_CA_4_validation,0.386724,0.357901,0.354373,0.360269,0.437514,0.438374,0.509599,0.411207,0.435104,0.417571,0.411910,0.415641,0.455774,0.415168,0.381999,0.395897,0.410331,0.433561,0.461866,0.497634,0.499692,0.404780,0.384837,0.380260,0.386451,0.437750,0.480305,0.496458
4,FOODS_1_001_TX_1_validation,0.156815,0.154247,0.146857,0.148227,0.143604,0.152647,0.199261,0.495317,0.429487,0.407307,0.474067,0.498414,0.487310,0.416363,0.371270,0.453230,0.421283,0.357992,0.423106,0.391395,0.380849,0.319272,0.307980,0.286163,0.285441,0.315941,0.371353,0.360824
5,FOODS_1_001_TX_2_validation,0.486281,0.452008,0.453362,0.418195,0.444328,0.516078,0.571664,0.471057,0.476016,0.448337,0.535173,0.541383,0.617415,0.561453,0.495011,0.468590,0.497406,0.479629,0.563357,0.595603,0.626785,0.475421,0.460132,0.457026,0.461418,0.518177,0.573148,0.560266
6,FOODS_1_001_TX_3_validation,0.381494,0.349582,0.351888,0.409011,0.444461,0.436243,0.572998,0.469635,0.523695,0.477985,0.475466,0.527255,0.564284,0.486344,0.462070,0.446737,0.467260,0.497705,0.516720,0.559020,0.527333,0.454462,0.426096,0.439446,0.437621,0.483934,0.519808,0.498346
7,FOODS_1_001_WI_1_validation,0.330306,0.381879,0.360334,0.367939,0.463430,0.777121,0.569313,0.453658,0.423427,0.412066,0.506687,0.584479,0.752757,0.585702,0.534487,0.542563,0.564871,0.570264,0.685310,0.827175,0.742336,0.546114,0.562854,0.575706,0.586001,0.713773,0.986732,0.852280
8,FOODS_1_001_WI_2_validation,0.306394,0.328072,0.346187,0.334537,0.415556,0.415769,0.447655,0.416000,0.466556,0.394071,0.413917,0.502114,0.512880,0.417134,0.419159,0.383921,0.473636,0.467210,0.480253,0.513331,0.476632,0.382657,0.382003,0.383902,0.380350,0.421674,0.439309,0.420076
9,FOODS_1_001_WI_3_validation,0.233811,0.232054,0.217484,0.221063,0.276180,0.352146,0.381389,0.363648,0.401541,0.356640,0.380423,0.440740,0.460509,0.381935,0.352688,0.357172,0.376674,0.360423,0.429813,0.468320,0.421977,0.339312,0.330829,0.342074,0.341507,0.400891,0.439542,0.386746


In [45]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [46]:
sub.shape

(60980, 29)

In [ ]:
#!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m "20200627(1) First public notebook under 0.50 added NBADurs and Pentecosts"; 0.51653 